<a href="https://colab.research.google.com/github/woo1027/chinese_bert/blob/main/chinese_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt


In [ ]:
toxic = pd.read_csv("/content/drive/My Drive//toxic.csv")
toxic.head()

In [ ]:
print("訓練集遺失值：\n{}".format(toxic.isnull().sum()))

In [ ]:
toxic = toxic.dropna(axis=0)


In [ ]:
newdata = toxic.drop(columns=['Unnamed: 0'])
newdata

clean_com_chinese欄位 為 翻譯成中文的評論

In [ ]:
label_stats = newdata[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum()
print(label_stats)

comments=newdata['clean_com_chinese'].to_list()
comments[:5]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Split data into training, testing sets & validation sets
train_data, test_data, train_labels, test_labels = train_test_split(
    comments, newdata.iloc[:,2:8].values, test_size=0.3, random_state=1)

#validation set
test_data, val_data, test_labels, val_labels = train_test_split(
    test_data, test_labels, test_size=0.67, random_state=1)

print('numbers of training Dataset ',len(train_data))
print('numbers of testing Dataset',len(test_data))
print('numbers of validation Dataset',len(val_data))

In [ ]:
def tokenize_and_encode(tokenizer, comments, labels, max_length=128):
    # Initialize empty lists to store tokenized inputs and attention masks
    input_ids = []
    attention_masks = []

    # Iterate through each comment in the 'comments' list
    for comment in comments:
        # Tokenize and encode the comment using the BERT tokenizer
        encoded_dict = tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Append the tokenized input and attention mask to their respective lists
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists of tokenized inputs and attention masks to PyTorch tensors
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    # Convert the labels to a PyTorch tensor with the data type float32
    labels = torch.tensor(labels, dtype=torch.float32)

    # Return the tokenized inputs, attention masks, and labels as PyTorch tensors
    return input_ids, attention_masks, labels

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

model =model.to(device)

In [ ]:
# Token Initialization
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese', do_lower_case=True)

# Model Initialization
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=6)

In [ ]:
# Tokenize and Encode the comments and labels for the training set
input_ids, attention_masks, labels = tokenize_and_encode(
    tokenizer,
    train_data,
    train_labels
)

# Step 4: Tokenize and Encode the comments and labels for the test set
test_input_ids, test_attention_masks, test_labels = tokenize_and_encode(
    tokenizer,
    test_data,
    test_labels
)

# Tokenize and Encode the comments and labels for the validation set
val_input_ids, val_attention_masks, val_labels = tokenize_and_encode(
    tokenizer,
    val_data,
    val_labels
)

In [ ]:
# Creating DataLoader for the balanced dataset
batch_size = 32
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#test
test_dataset = TensorDataset(test_input_ids, test_attention_masks, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#val
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load the tokenizer and model from the saved directory
model_name ="/content/drive/My Drive/Saved_model"
Bert_Tokenizer = BertTokenizer.from_pretrained(model_name)
Bert_Model = BertForSequenceClassification.from_pretrained(model_name).to(device)

In [ ]:
def predict_user_input(input_text, model=Bert_Model, tokenizer=Bert_Tokenizer,device=device):
    user_input = [input_text]

    user_encodings = tokenizer(user_input, truncation=True, padding=True, return_tensors="pt")

    user_dataset = TensorDataset(user_encodings['input_ids'], user_encodings['attention_mask'])

    user_loader = DataLoader(user_dataset, batch_size=1, shuffle=False)

    model.eval()
    with torch.no_grad():
        for batch in user_loader:
            input_ids, attention_mask = [t.to(device) for t in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)
    predicted_labels = (predictions.cpu().numpy() > 0.5).astype(int)
    return predicted_labels[0].tolist()

In [ ]:
text = '你是智障嗎?'
predict_user_input(input_text=text)

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

In [ ]:
text = "你真的很討厭，但我愛你"
predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text,
                   device=device)

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

In [ ]:
input_text = '你這個婊子'
predict_user_input(input_text)

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨

In [ ]:
text = "閉嘴，笨蛋"
predict_user_input(model=Bert_Model,
                   tokenizer=Bert_Tokenizer,
                   input_text=text,
                   device=device)

有毒的、嚴重有毒、猥褻、威脅、侮辱、身分仇恨